Download pretrained model 
https://ai2-s2-research.s3-us-west-2.amazonaws.com/longformer/longformer-large-4096.tar.gz



In [3]:
#!conda create --name longformer python=3.7
#!conda activate longformer
!conda install cudatoolkit=10.0
!pip install git+https://github.com/allenai/longformer.git

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - cudatoolkit=10.0

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


  Cloning https://github.com/allenai/longformer.git to /private/var/folders/r3/zbc0j5755cd424h_f2g7b53h0000gr/T/pip-req-build-ipk36ze8
  Running command git clone -q https://github.com/allenai/longformer.git /private/var/folders/r3/zbc0j5755cd424h_f2g7b53h0000gr/T/pip-req-build-ipk36ze8
  Cloning http://github.com/ibeltagy/transformers.git

In [6]:
import torch
from longformer.longformer import Longformer, LongformerConfig
from longformer.sliding_chunks import pad_to_window_size
from transformers import RobertaTokenizer

config = LongformerConfig.from_pretrained('longformer-large-4096/') 
# choose the attention mode 'n2', 'tvm' or 'sliding_chunks'
# 'n2': for regular n2 attantion
# 'tvm': a custom CUDA kernel implementation of our sliding window attention
# 'sliding_chunks': a PyTorch implementation of our sliding window attention
config.attention_mode = 'sliding_chunks'

model = Longformer.from_pretrained('longformer-large-4096/', config=config)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer.model_max_length = model.config.max_position_embeddings

SAMPLE_TEXT = ' '.join(['Hello world! '] * 1000)  # long input document

input_ids = torch.tensor(tokenizer.encode(SAMPLE_TEXT)).unsqueeze(0)  # batch of size 1

# TVM code doesn't work on CPU. Uncomment this if `config.attention_mode = 'tvm'`
# model = model.cuda(); input_ids = input_ids.cuda()

# Attention mask values -- 0: no attention, 1: local attention, 2: global attention
attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device) # initialize to local attention
attention_mask[:, [1, 4, 21,]] =  2  # Set global attention based on the task. For example,
                                     # classification: the <s> token
                                     # QA: question tokens

# padding seqlen to the nearest multiple of 512. Needed for the 'sliding_chunks' attention
input_ids, attention_mask = pad_to_window_size(
        input_ids, attention_mask, config.attention_window[0], tokenizer.pad_token_id)

output = model(input_ids, attention_mask=attention_mask)[0]
output

Downloading: 100%|██████████| 899k/899k [00:00<00:00, 2.42MB/s]
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 979kB/s]


In [7]:
output

tensor([[[-0.0414,  0.0574, -0.5770,  ..., -0.2942, -0.2785,  0.1179],
         [ 0.2158,  0.3126, -0.9947,  ..., -0.8295, -0.2143,  0.0195],
         [ 0.3286,  0.4128, -0.2817,  ...,  0.4203, -0.5912,  0.6113],
         ...,
         [-0.1238,  0.3869,  0.2201,  ..., -0.6698,  0.5555,  0.0536],
         [-0.1238,  0.3869,  0.2201,  ..., -0.6698,  0.5555,  0.0536],
         [-0.1238,  0.3869,  0.2201,  ..., -0.6698,  0.5555,  0.0536]]],
       grad_fn=<NativeLayerNormBackward>)